In [1]:
## import the req libraries
import numpy as np
import tensorflow as tf
import ipywidgets as widgets
from PIL import Image
import h5py
import io
from ipywidgets import VBox, Layout
from tensorflow.keras.preprocessing.image import apply_affine_transform
import time
from tensorflow.keras.models import model_from_json

from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

##  <h1><center>VGG Face Descriptor Model based Gender Classifier</center></h1>

<center><img src="head_mast.png"></center>

This web app outputs the probability of a subject in a face centered image being Female. The model was only trained to identify between the genders - female and male. The user can also choose the number of Test Time Augmentation (T.T.A.) to get more accurate predictions.

### Directions For Use:
1. Select the desired # of Test Time Augmentations using the slider.
2. Upload a face centered image of a person.

### Outputs
1. The user, in realtime, will see the test time augmentations being applied and the corresponding model prediction.
2. At the end, The final prediction (avergae of all TTA predictions) is displayed along with the original image uploaded.

This is a proof of concept for research purpose ONLY. The model can be used in identification or tagging systems for returning faster query search results by reducing the search space on gender.

In [14]:
## download model weights
! python download_gdrive.py 18q2BaRaDNJ7ChLSHm58VV0c0C9h46FT8 vggFaceGender.zip

In [ ]:
!unzip vggFaceGender.zip

In [3]:
## load the model
model = tf.keras.models.load_model("vggFaceGender")

In [4]:
### widgets
##upload button
btn_upload = widgets.FileUpload(accept ='image/*'
                                , multiple = True
                                , align_items='center'
                                ,layout=Layout(margin='auto'))

In [5]:
## slider for tta
sldr = widgets.IntSlider(value=5
                         , min=1
                         , max=10
                         , step=1
#                         , description='T.T.A.:'
                         , disabled=False
                         , continuous_update=False
                         , orientation='horizontal'
                         , align_items='center'
                         , readout=True
                         , readout_format='d'
                         , layout=Layout(margin='auto'))

In [6]:
## label predictor
out = widgets.Output(layout=Layout(margin='auto'))
lbl_pred = widgets.Label(layout=Layout(margin='auto'))

In [7]:
## the onclick event for upload button
def on_click(change):
    img = widgets.Image(value = btn_upload.data[-1])
    out.clear_output()
    with out: display(img)
    image1 = Image.open(io.BytesIO(btn_upload.data[-1]))
    image = np.array(image1)
    aug_img = []
    pred_ls =[]
    for i in range(sldr.value):
        ## convert to tensor 
        image = tf.convert_to_tensor(image, tf.float32)
        image = tf.image.resize(image, [224,224])
        image = np.array(image)
        ## rotation 
        transformation = apply_affine_transform(image
                                                , theta = 40 * (np.random.normal(0, 1, 1)[0]))
        img1 = tf.image.random_flip_left_right(transformation)
        
        img1 = tf.image.random_hue(img1, 0.02)
        img1 = tf.image.random_saturation(img1, 0.7, 1.3)
        #img1 = tf.image.random_contrast(img1, 0.8, 1.2)
        img1 = tf.image.random_brightness(img1, 0.1)
        #img1 = tf.image.random_crop(img1, [250, 250, 3])

        img1 = tf.cast(img1, tf.float32) / 255.0
        input_arr = tf.image.resize(img1, [224,224])
        input_arr = tf.reshape(input_arr, [224,224, 3])
        
        out.clear_output()
        img2 = Image.fromarray(((np.array(input_arr)*255).astype(np.int8)), 'RGB')
        with out: display(img2)
        input_arr = tf.expand_dims(input_arr, axis=0)
        pred = model.predict(input_arr, batch_size=1)[0][0]
        pred_ls.append(1-pred)
        lbl_pred.value = f'T.T.A. # {i+1} Prediction :{ str(round(1-pred,2))}'
        # Wait for 5 seconds
        time.sleep(2)
    out.clear_output()
    img = image1.resize((224,224))
    with out: display(img)

    ans = str(round(np.mean(pred_ls),2))    
    lbl_pred.value = f'Final Result - Probability of Subject being Female :{ ans}'

In [8]:
btn_upload.observe(on_click, names=['data'])

In [9]:
## organize widgets
display(VBox([widgets.Label('Select the # Test Time Augmentations ( T.T.A. ) using the slider',layout=Layout(margin='auto'))
              , sldr
              , widgets.Label('Upload your image',layout=Layout(margin='auto'))
              , btn_upload
              , out
              ,lbl_pred
              ]
             , layout=Layout(border='solid'
                             ,width='50%'
                             ,margin='auto'))
        )